<a href="https://colab.research.google.com/github/shruthireddyrekula/Sentiment_analysis_IMDB_TF/blob/main/IMDB_Review_Transformer_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers


In [ ]:
import tensorflow_datasets as tfds

(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised=True,
    with_info=True
)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(sentences, tokenizer, max_length):
    return tokenizer(sentences, truncation=True, padding='max_length', max_length=max_length, return_tensors="tf")

train_tokens = tokenize_data([s.numpy().decode('utf8') for s, l in train_data], tokenizer, 256)
test_tokens = tokenize_data([s.numpy().decode('utf8') for s, l in test_data], tokenizer, 256)

In [ ]:
from transformers import TFBertModel
import tensorflow as tf

def build_model(transformer, max_length):
    input_word_ids = tf.keras.Input(shape=(max_length,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

transformer_layer = TFBertModel.from_pretrained('bert-base-uncased')
model = build_model(transformer_layer, 256)

In [ ]:
train_labels = [l.numpy() for s, l in train_data]
test_labels = [l.numpy() for s, l in test_data]
train_labels_tensor = tf.convert_to_tensor(train_labels)
test_labels_tensor = tf.convert_to_tensor(test_labels)
history = model.fit(train_tokens['input_ids'], train_labels_tensor, validation_data=(test_tokens['input_ids'], test_labels_tensor), batch_size=12, epochs=10,verbose=2)

In [ ]:
model.evaluate(test_tokens['input_ids'], test_labels_tensor)